In [1]:
import pandas as pd

import glob

from google.oauth2 import service_account
# from oauth2client.service_account import ServiceAccountCredentials

In [2]:
voter_history_directory = r'C:\Users\thewh\Documents\Master Voting History'

# Google Service Credential JSON <- You'll need your own!
# gs_credentials = ServiceAccountCredentials.from_json_keyfile_name('cpc-datawarehouse-51210-cbbf1375a84e.json')
# Establish BigQuery credentials
bq_credentials = service_account.Credentials.from_service_account_file(
    'cpc-datawarehouse-51210-cbbf1375a84e.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

bq_project_id = 'cpc-datawarehouse-51210'
bq_table_id = 'cpc-datawarehouse-51210.co_voterfile.vote-history'

bq_schema_list = [
    {'name': 'VOTER_ID', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'ELECTION_TYPE', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'ELECTION_DATE', 'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'ELECTION_DESCRIPTION', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'VOTING_METHOD', 'type': 'STRING', 'mode': 'REQUIRED'},
    {'name': 'PARTY', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'COUNTY_NAME', 'type': 'STRING', 'mode': 'NULLABLE'}
]

In [3]:
# Load vote history data frame
print("Beginning to assemble vote history file.")

allfiles = glob.glob(voter_history_directory + "/*.txt")
history_df = pd.DataFrame()
_lst = []
for _file in allfiles:
    _df = pd.read_csv(_file, sep=',', encoding='cp437', index_col=None, header=0, low_memory=False)
    _lst.append(_df)
history_df = pd.concat(_lst)

history_df.reset_index(drop=True, inplace=True)

Beginning to assemble vote history file.


In [4]:
for column in list(history_df):
    print(column)
    history_df[column] = history_df[column].apply(str)

print("Datetime")
history_df['ELECTION_DATE'] = pd.to_datetime(history_df['ELECTION_DATE'], format='%m/%d/%Y', infer_datetime_format=True)

VOTER_ID
ELECTION_TYPE
ELECTION_DATE
ELECTION_DESCRIPTION
VOTING_METHOD
PARTY
COUNTY_NAME
Datetime


In [5]:
history_df.to_gbq(destination_table=bq_table_id, project_id=bq_project_id, if_exists='replace', table_schema=bq_schema_list, credentials=bq_credentials)

100%|██████████| 1/1 [00:00<00:00, 499.68it/s]


In [6]:
del history_df